In [1]:
import requests
import csv
import json
from datetime import datetime
import time

In [2]:
service_token = "))"  #сервисный токен, для конфеденциальности удалила после прогона кода
domain = "id267391840" 
api_version = "5.131"
output = "vk_posts_statistics.csv"

In [6]:
def get_wall_posts(domain, count=100):
    url = "https://api.vk.com/method/wall.get"
    params = {
        'domain': domain,
        'count': count,
        'access_token': service_token,
        'v': api_version,
        'extended': 0
    }
    
    response = requests.get(url, params=params)
    data = response.json()
    posts = data.get('response', {}).get('items', [])
    return posts

def parse_posts(posts):
    parsed_data = []
    
    for post in posts:
        if post.get('copy_history'):
            continue
            
        post_date = datetime.fromtimestamp(post['date'])
        likes_count = post.get('likes', {}).get('count', 0)
        post_text = post.get('text', '')
        
        parsed_data.append({
            'post_date': post_date.strftime('%Y-%m-%d %H:%M:%S'),
            'likes_count': likes_count,
            'post_text': post_text,
            'hour': post_date.hour,
            'day_of_week': post_date.strftime('%A'),
            'weekday_number': post_date.weekday()
        })
    
    return parsed_data

def save_to_csv(data, filename):
    with open(filename, 'w', newline='', encoding='utf-8-sig') as csvfile:  
        fieldnames = ['post_date', 'likes_count', 'post_text', 'hour', 'day_of_week', 'weekday_number']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, quoting=csv.QUOTE_ALL)    
        writer.writeheader()
        for row in data:
            writer.writerow(row)

In [9]:
posts = get_wall_posts(domain, count=100)
parsed_data = parse_posts(posts)
save_to_csv(parsed_data, output)